<a href="https://colab.research.google.com/github/allseenn/neuronet/blob/main/hw03c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в нейронные сети

## Урок 3. TensorFlow

### Домашнее задание

1. Попробуйте улучшить работу нейронной сети (разобранную на уроке), обучавшейся на датасет Fashion-MNIST. Напишите в комментариях к уроку, какого результата вы добились от нейросети и что помогло улучшить её точность
2. Поработайте с документацией TensorFlow 2. Попробуйте найти полезные команды TensorFlow, неразобранные на уроке
3. \* Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews.

In [7]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
# Загрузка данных
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [9]:
# Нормализация данных и изменение размера
x_train = x_train.reshape((-1, 28, 28, 1)) / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)) / 255.0

In [10]:
# Преобразование меток в формат one-hot encoding
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [11]:
# Создание модели
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [12]:
# Компиляция модели с оптимизатором Adam
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Создание аугментации данных
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [24]:
# Обучение модели с использованием аугментированных данных
model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
                    steps_per_epoch=len(x_train) / 128, epochs=20,
                    validation_data=(x_test, y_test))

Epoch 1/20
  1/468 [..............................] - ETA: 35s - loss: 0.2373 - accuracy: 0.9062

<ipython-input-24-84c683495281>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),


468/468 [==============================] - 19s 41ms/step - loss: 0.3020 - accuracy: 0.8914 - val_loss: 0.2367 - val_accuracy: 0.9184
Epoch 2/20
468/468 [==============================] - 19s 41ms/step - loss: 0.3063 - accuracy: 0.8898 - val_loss: 0.2373 - val_accuracy: 0.9147
Epoch 3/20
468/468 [==============================] - 18s 39ms/step - loss: 0.3019 - accuracy: 0.8906 - val_loss: 0.2389 - val_accuracy: 0.9157
Epoch 4/20
468/468 [==============================] - 19s 40ms/step - loss: 0.2995 - accuracy: 0.8924 - val_loss: 0.2294 - val_accuracy: 0.9200
Epoch 5/20
468/468 [==============================] - 18s 39ms/step - loss: 0.2975 - accuracy: 0.8917 - val_loss: 0.2376 - val_accuracy: 0.9157
Epoch 6/20
468/468 [==============================] - 19s 41ms/step - loss: 0.3022 - accuracy: 0.8898 - val_loss: 0.2527 - val_accuracy: 0.9131
Epoch 7/20
468/468 [==============================] - 18s 38ms/step - loss: 0.3020 - accuracy: 0.8909 - val_loss: 0.2357 - val_accuracy: 0.9190
Epo

In [25]:
# Оценка модели
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.22187510132789612
Test accuracy: 0.9218999743461609


In [26]:
import numpy as np
from sklearn.metrics import classification_report

# Получить прогнозы вероятностей модели на тестовых данных
y_pred_probabilities = model.predict(x_test)

# Преобразовать вероятности в индексы классов
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Преобразовать метки в индексы классов
y_test_classes = np.argmax(y_test, axis=1)

# Сформировать отчет классификации для каждого класса
class_report = classification_report(y_test_classes, y_pred)
print("Classification Report:")
print(class_report)

313/313 [==============================] - 1s 2ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.86      0.88      1000
           1       0.99      0.98      0.99      1000
           2       0.89      0.88      0.89      1000
           3       0.90      0.93      0.92      1000
           4       0.86      0.89      0.87      1000
           5       0.98      0.98      0.98      1000
           6       0.77      0.77      0.77      1000
           7       0.95      0.98      0.96      1000
           8       0.99      0.99      0.99      1000
           9       0.98      0.96      0.97      1000

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



Как показали эксперементы, архитектура сверточной сети в комбинации с полносвязными слоями, показала хорошие результаты. Рекомендуется обучать сетку не более чем на 30 эпохах, т.к. далее сеть перестает эффективно обучаться.
Практически полностью код использует keras, кроме последнего блока оценки классов. По факту использовал знания полученные с 4 семинара, сверточные сети и немного упоминалось в 5 семинаре про аугментацию, применил ее не совсем понимая смысл происходящего :) Сеть имеет лучшие показатели, но ощутимее дольше обучается, используя мощности GPU, возможно из-за того, что используется чистый keras. Заметил на семинаре, что к недостаткам кераса, относят неспособность полноценного использования Multy GPU, в отличие от tensorflow.